In [37]:
import os
from chatgpt_helper import GPTHelper
from tqdm import tqdm
import os
from config import persona_prompt, character_prompt, merge_prompt, scene_prompt, zh_persona_prompt, zh_character_prompt, zh_merge_prompt

gpt_helper = GPTHelper()
print("gpt_helper: ", gpt_helper)

gpt_helper:  GPTHelper(api_key: 'sk-npSRVdUhy...b8309DbF5E569', base_url: 'https://api.ai-gaochao.cn/v1', model: 'gpt-4o', temperature: 1.0, max_tokens: 128000, top_p: 1.0, frequency_penalty: 0.1, presence_penalty: 0.2, max_output_tokens: 4096, prompt_ratio: 0.8)


In [2]:
print(zh_persona_prompt),print(zh_character_prompt), print("merge_prompt: ", zh_merge_prompt)

## **任务：**
分析提供的A和B之间的双人对话，评估 **B** 的个性特征（基于五大性格特质维度）。

对于B的每个个性特征：
1.  **评分**：独立地给每个维度分配“高”、“低”或“不确定”的分数。
2.  **结论**：以对B的高层次描述开头，例如“B是一个[特质描述]的人”，然后是简洁的支持细节。
3.  **原因**：用对话中的具体证据（**可引述A或B的发言**）来证明所分配分数是正确的。
4.  只有当对话中没有明确证据时，才将特质标记为**不确定**。在这种情况下，完全省略“结论”和“原因”字段。

**个性特质维度：**

1.  **尽责性**：调节冲动控制以从事目标导向行为的能力。
    * **低分者**：无组织、粗心、拖延、缺乏纪律、冲动、难以完成任务、不喜欢结构、从事粗心行为。
    * **高分者**：有条理、有纪律、注重细节、深思熟虑、细心、有能力、尽职、追求成就、自律、体贴、善于完成任务和实现目标。

2.  **宜人性**：反映个人如何对待人际关系以及如何与他人互动。
    * **低分者**：多疑、苛刻、侮辱性、固执、爱炫耀、不同情、不关心、多疑、操纵、不合作、对抗性、不太可能被信任。
    * **高分者**：信任他人、坦率、利他、顺从、谦逊、同情、善解人意、心地善良、乐于助人、合作、受欢迎、对他人需求敏感。

3.  **外向性**：反映寻求与环境（特别是社会环境）互动的倾向和强度。
    * **低分者**：喜欢独处、社交互动会感到疲惫、善于反思、不喜欢成为关注中心、内向、安静、喜欢倾听他人、需要独处来恢复精力。
    * **高分者**：善于交际、社交互动会感到精力充沛、寻求刺激、乐于成为关注中心、外向、自信、热爱玩乐、在社交场合如鱼得水、乐于表达意见。

4.  **开放性**：指尝试新事物和从事富有想象力或智力活动的意愿。
    * **低分者**：可预测、不富于想象力、不适应变化、严格遵守常规、传统、喜欢熟悉的事物而非未知事物、务实、难以进行创造性或抽象思考。
    * **高分者**：好奇、富有想象力、有创造力、乐于尝试新事物、不墨守成规、有艺术性、喜欢多样性、重视独立性、喜欢旅行和学习新事物。

5.  **神经质**：描述整体情绪稳定性和经历负面情绪的可能性。
    * **低分者

(None, None, None)

In [3]:
from typing import List, Dict

def transfer_utterances_to_string(dialog_lst: List[Dict], order="A-B") -> str:
    """将对话转换为字符串格式"""
    return_str = ""
    if order == "A-B":
        user_role = "A"
        assistant_role = "B"
    elif order == "B-A":
        user_role = "B"
        assistant_role = "A"
    else:
        raise ValueError(f"不支持的对话顺序: {order}")
    for dialog in dialog_lst:
        if dialog['role'] == "user":
            return_str += f"[{user_role}]: {dialog['content']}\n"
        elif dialog['role'] == "assistant":
            return_str += f"[{assistant_role}]: {dialog['content']}\n"
    return return_str.strip()

In [4]:
data = {"bot_persona": ["B1: 我 不过 任何 节日", "B2: 我 肤色 很 白", "B3: 我 喜欢 看 篮球", "B4: 帮助 别人 使 我 快乐"], "user_said_persona": ["U1: 我 有 一 条狗", "U2: 我 想 去 跳伞", "U3: 我 叫 小 夏", "U4: 我 是 女生"], "user_no_said_persona": ["U5: 我 喜欢 玩 电子 游戏"], "conversation": [{"role": "user", "content": "小度，你在么？"}, {"role": "assistant", "content": "在的，之前你不是计划去跳伞吗？去了吗？"}, {"role": "user", "content": "没有，最近我的男朋友生病了，想去也没办法去了，以后再说吧。"}, {"role": "assistant", "content": "哦，原来如此啊，那你男朋友身体好些了吗？"}, {"role": "user", "content": "他现在好多了，还能跟我一起打游戏。"}, {"role": "assistant", "content": "哇，你们两个人好合拍，相比之下我这孤家寡人只好一个人看篮球比赛啦。"}, {"role": "user", "content": "他也很喜欢看篮球比赛，每次他在家看比赛，我就去遛狗。"}, {"role": "assistant", "content": "哈哈，看来你对篮球比赛不感兴趣啊？对了，你的狗是什么品种的呢？"}, {"role": "user", "content": "我养了一只白色的泰迪，超级可爱！"}, {"role": "assistant", "content": "哇，我也喜欢泰迪，超可爱的！"}, {"role": "user", "content": "我给我的小狗起名叫贝贝。"}, {"role": "assistant", "content": "这个名字挺中性，你家狗狗是公的还是母的呢？"}, {"role": "user", "content": "它是一只小母狗。"}, {"role": "assistant", "content": "送的还是买的呢？我也想养一只呢。"}, {"role": "user", "content": "我在狗舍买的，品相很好。"}, {"role": "assistant", "content": "嗯嗯，有空我也去看看，寻一只缘的狗狗回来养。"}]}

dialogue_data = data["conversation"]
data = transfer_utterances_to_string(dialogue_data)
print(data)


[A]: 小度，你在么？
[B]: 在的，之前你不是计划去跳伞吗？去了吗？
[A]: 没有，最近我的男朋友生病了，想去也没办法去了，以后再说吧。
[B]: 哦，原来如此啊，那你男朋友身体好些了吗？
[A]: 他现在好多了，还能跟我一起打游戏。
[B]: 哇，你们两个人好合拍，相比之下我这孤家寡人只好一个人看篮球比赛啦。
[A]: 他也很喜欢看篮球比赛，每次他在家看比赛，我就去遛狗。
[B]: 哈哈，看来你对篮球比赛不感兴趣啊？对了，你的狗是什么品种的呢？
[A]: 我养了一只白色的泰迪，超级可爱！
[B]: 哇，我也喜欢泰迪，超可爱的！
[A]: 我给我的小狗起名叫贝贝。
[B]: 这个名字挺中性，你家狗狗是公的还是母的呢？
[A]: 它是一只小母狗。
[B]: 送的还是买的呢？我也想养一只呢。
[A]: 我在狗舍买的，品相很好。
[B]: 嗯嗯，有空我也去看看，寻一只缘的狗狗回来养。


In [5]:
flag, character_output = gpt_helper.request_text_api(user_input=data, system_messages=zh_character_prompt)
print(character_output)

```json
{
    "name": "小度",
    "gender": [],
    "age": [],
    "location": [],
    "occupation": [],
    "education": [],
    "family_relationships": [],
    "routines_or_habits": ["在看篮球比赛"],
    "goals_or_plans": ["想养一只属于自己的泰迪狗"],
    "social_relationships": ["与A是朋友关系，会分享个人兴趣和想法。"],
    "language_style": ["使用轻松的聊天风格（“孤家寡人”、“超可爱的”）"],
    "other_experiences": ["目前没有宠物，计划“寻一只缘的狗狗回来养”"],
    "interlocutor_info": {
        "name_or_title": "",
        "relationship": "朋友"
    }
}
```


In [24]:
flag, persona_output = gpt_helper.request_text_api(user_input=data, system_messages=zh_persona_prompt)
print(persona_output)

```json
{
    "尽责性": {
        "score": "不确定"
    },
    "宜人性": {
        "score": "高",
        "conclusion": "B是一个关心他人并乐于表现出对他人兴趣的人。",
        "reason": "B主动询问A男朋友的健康状况（‘那你男朋友身体好些了吗？’）和狗的品种（‘对了，你的狗是什么品种的呢？’），表现出对A生活细节的关注和关心。"
    },
    "外向性": {
        "score": "低",
        "conclusion": "B是一个偏内向的人，更倾向于独自活动。",
        "reason": "B提到自己是‘孤家寡人只好一个人看篮球比赛’，并没有寻求社交活动或与他人一起参与。"
    },
    "开放性": {
        "score": "高",
        "conclusion": "B是一个对新体验持开放态度的人，对养狗的想法感到兴趣。",
        "reason": "B很感兴趣地讨论关于狗的话题，并表示‘有空我也去看看，寻一只缘的狗狗回来养’，显示出愿意尝试新事物的态度。"
    },
    "神经质": {
        "score": "不确定"
    }
}
```


In [26]:
import re
import json
import random

def extract_json_from_string(string: str) -> dict:
    """从字符串中提取JSON对象"""
    try:
        # 使用正则表达式提取JSON对象
        first_left_brace_index = string.find("{")
        last_right_brace_index = string.rfind("}")
        json_str = string[first_left_brace_index:last_right_brace_index+1]
        if json_str:
            json_str = json_str.strip()
            return json.loads(json_str)
        else:
            return None
    except Exception as e:
        print(f"提取JSON对象时发生错误: {e}")
        return None


def random_dict(input_dict: dict) -> dict:
    keys = list(input_dict.keys())
    random.shuffle(keys)
    return {key: input_dict[key] for key in keys}


def filter_character_output(character_output: str) -> str:
    json_obj = extract_json_from_string(character_output)
    character_name = json_obj.get("name","")
    if character_name:
        return_dict = {
            "name": character_name
        }
        json_obj.pop("name")
    else:
        return_dict = {}
    new_dict = {}
    for key, value in json_obj.items():
        if not value:
            pass
        else:
            new_dict[key] = value
    new_dict = random_dict(new_dict)
    return_dict.update(new_dict)
    interlocutor_info = return_dict.get("interlocutor_info","")
    if interlocutor_info:
        return_dict.pop("interlocutor_info")
    return return_dict, {"interlocutor_info":interlocutor_info}
    

def filter_persona_output(character_output: str) -> str:
    """过滤character_output中的无效信息"""
    json_obj = extract_json_from_string(character_output)
    new_return = {}
    for key, value in json_obj.items():
        if value and value.get("score","Inconclusive") == "Inconclusive":
            pass
        elif value and value.get("score","不确定") == "不确定":
            pass
        else:
            new_return[key] = value["conclusion"]
    return random_dict(new_return)

filtered_character_output,interlocutor_info = filter_character_output(character_output)
filtered_persona_output = filter_persona_output(persona_output)
filtered_character_output, filtered_persona_output, interlocutor_info



({'name': '小度',
  'routines_or_habits': ['在看篮球比赛'],
  'social_relationships': ['与A是朋友关系，会分享个人兴趣和想法。'],
  'goals_or_plans': ['想养一只属于自己的泰迪狗'],
  'other_experiences': ['目前没有宠物，计划“寻一只缘的狗狗回来养”'],
  'language_style': ['使用轻松的聊天风格（“孤家寡人”、“超可爱的”）']},
 {'外向性': 'B是一个偏内向的人，更倾向于独自活动。',
  '宜人性': 'B是一个关心他人并乐于表现出对他人兴趣的人。',
  '开放性': 'B是一个对新体验持开放态度的人，对养狗的想法感到兴趣。'},
 {'interlocutor_info': {'name_or_title': '', 'relationship': '朋友'}})

In [34]:
character_name = filtered_character_output.get("name","")
interlocutor_name = interlocutor_info["interlocutor_info"].get("name_or_title","")
if not interlocutor_name:
    interlocutor_name = "对方"

merge_input = f"""
**客观事实属性**
{filtered_character_output}
**主观特性属性**
{filtered_persona_output}

**当前对话场景**
{interlocutor_info}
"""

merge_input = merge_input.replace("A", interlocutor_name).replace("B", character_name)

print(merge_input)



**客观事实属性**
{'name': '小度', 'routines_or_habits': ['在看篮球比赛'], 'social_relationships': ['与对方是朋友关系，会分享个人兴趣和想法。'], 'goals_or_plans': ['想养一只属于自己的泰迪狗'], 'other_experiences': ['目前没有宠物，计划“寻一只缘的狗狗回来养”'], 'language_style': ['使用轻松的聊天风格（“孤家寡人”、“超可爱的”）']}
**主观特性属性**
{'外向性': '小度是一个偏内向的人，更倾向于独自活动。', '宜人性': '小度是一个关心他人并乐于表现出对他人兴趣的人。', '开放性': '小度是一个对新体验持开放态度的人，对养狗的想法感到兴趣。'}

**当前对话场景**
{'interlocutor_info': {'name_or_title': '', 'relationship': '朋友'}}



In [35]:
merge_prompt = """
# 用户画像属性综合与叙事生成

## 目标
将提供的结构化用户画像属性（键值对），综合并转化为一段**高度概括、连贯流畅的第二人称描述**。该描述将分为客观事实和主观特征两个段落。

## 输出结构

1.  **第一段：客观事实总结**
    * 本段聚焦于用户的**外部情况和背景框架**。
    * 内容应整合用户的职业、教育、地理位置、家庭构成和关键生活经历等**可验证信息**。
    * 目标是构建一个清晰、准确的外部画像，不包含主观推断。

2.  **第二段：主观特征刻画**
    * 本段侧重于用户的**内在特质和行为模式**。
    * 内容应综合分析用户的沟通风格、习惯、价值观和隐含的情绪或态度。
    * 目标是揭示用户的个性、思维方式以及与世界互动的方式。

3.  **第三段：描述当前场景**
    * 本段描述用户当前的对话场景，包括对话双方的姓名、关系等。

## 核心原则

1.  **高度概括与抽象 (High-Level Abstraction)**
    * **这是首要原则。** 生成的描述必须是高维度的总结，而非细节的罗列。应提炼信息的核心本质，避免直接复述具体的例子或话语。
    * **示例**：
        * **输入属性**: `{"other_experiences": ["提到‘很小的时候’父母离婚"]}`
        * **错误描述 (细节复述)**: `“在你很小的时候，你的父母离婚了。”`
        * **正确描述 (高度概括)**: `“你的成长经历中包含重要的家庭结构变化。”`

2.  **第二人称视角 (Second-Person Perspective)**
    * 全文必须使用“你”和“你的”来进行叙述，以直接对话的口吻呈现。

3.  **信息完整性 (Completeness of Information)**
    * 必须准确地反映所提供的**所有**键值对属性，确保没有核心信息遗漏。

4.  **流畅与精炼 (Fluency and Conciseness)**
    * 段落内部和段落之间应有自然的逻辑流，文字表达要流畅、精炼，避免生硬堆砌和冗余。

## 示例

**客观事实属性**
{
    "name": "小李",
    "occupation": ["在科技公司担任软件工程师"],
    "education": ["拥有计算机科学硕士学位"],
    "location": ["目前居住在上海"],
    "family_relationships": ["最近刚结婚"],
    "routines_or_habits": ["有定期健身的习惯", "业余时间喜欢研究开源项目"],
}

**主观特性属性**
{
  "language_style": ["沟通直接、以解决问题为导向", "在技术讨论中表现出自信"],
  "尽责性": "是一个非常尽职尽责、注重细节的人，在工作中追求高标准。",
  "宜人性": "是一个谦逊、合作且懂得感恩的人，在团队中表现出良好的互动姿态。",
  "外向性": "在专业环境中表现出自信，乐于承担需要公开展示的任务。",
  "开放性": "是一个乐于学习并应用新知识、对新观点持开放态度的人。",
  "神经质": "表现出情绪稳定和自信，能够沉着地接受赞扬和挑战。"
}

**当前对话场景**
"interlocutor_info": {
    "name_or_title": "小王",
    "relationship": "朋友"
}


**预期输出 (生成的描述):**

你叫小李，是一名base在上海的软件工程师，拥有计算机科学的硕士教育背景，并在此从事软件工程师的专业工作。在个人生活层面，你最近刚刚完成了婚姻这件人生大事，进入了家庭生活的新阶段。你的日常生活体现了明确的结构性和目的性：既通过定期健身来关注身体状态，也投入业余时间探索开源项目，保持着对技术领域持续的个人追求。

你的内在特质表现为一个沉稳、自驱的专业人士。沟通时，你直接坦诚，以解决问题为核心，在专业技术领域尤其能展现出自信。这种自信源于你稳定从容的情绪内核和高度的尽责心，后者驱使你追求卓越，并对细节一丝不苟。在团队互动中，你又能展现出谦逊、合作与感恩的态度。你对新知识与新观点抱持着开放的心态，这构成了你不断学习和自我提升的内在驱动力。

你正在与你的朋友小王聊天。
"""

In [36]:
flag, merge_output = gpt_helper.request_text_api(user_input=merge_input, system_messages=merge_prompt)
print(merge_output)


你叫小度，目前正在关注篮球比赛，并时常与朋友分享个人兴趣和想法。你怀有养一只泰迪狗的计划，目前还没有宠物，因此希望能够找到命中注定的小伙伴。你的沟通风格轻松随意，喜欢用“孤家寡人”、“超可爱的”这样的话语拉近与对方的距离。

性格上，你偏向内向，喜欢独自活动，但乐于关心他人的需求和表现出对他人兴趣的关注。对于新体验持开放态度的你，对养狗充满了期待，这种开放性为你带来了好奇和冒险精神。

你目前正在和一个朋友聊天。
